# Scientific Programming

[LV 707716](https://orawww.uibk.ac.at/public/lfuonline_lv.details?sem_id_in=18W&lvnr_id_in=707716)

**[Fabien Maussion](http://fabienmaussion.info)**

<img align="left" width="30%" src="https://www.uibk.ac.at/public-relations/grafik_design/images/logo/download/sublogos/institute-sublogos/atmospheric-and-cryospheric-sciences.png"/>

## How to use these slides

- ``<space>`` : go to the next slide
- ``<shift+space>``: go back
- ``<left right up down arrows>``: navigate through the presentation structure
- ``<esc>``: toggle overview mode

<a href="https://creativecommons.org/licenses/by-nc-sa/4.0/" target="_blank">
  <img align="left" src="http://mirrors.creativecommons.org/presskit/buttons/88x31/svg/by-nc-sa.eu.svg"/>
</a>

<br>
<br>
        
These lecture notes and exercises are licensed under a [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License](https://creativecommons.org/licenses/by-nc-sa/4.0/).

Feel free to use / adapt them, but don't sell them, and share them under the same licence.

# Organizational

# Exam

**What?** Will be an **oral exam** after all: 30 min preparation, 30 questions.

**When?** first week or last week of February.
Choose your examination time slot on our google sheet (first come, first serve).

## How will the exam look like? 

- we meet in my office
- topic + list of questions assigned randomly at arrival 
- 30 minutes preparation: I will provide a computer (linux with internet)
- 30 minutes questions

## What will be tested for?

- ability to read and understand code from freely available scientific software
- ability to explain a code structure / algorithm
- knowledge of the core concepts and semantics learned during the lecture 
- ability to solve a programming problem and plan a programming task

If you worked regularly during the semester and learned the **bold** elements in the lecture notes you should be good to go!

## Program for the last weeks

- **Today**: scientific workflow, code documentation (should've been much earlier...)
- **Next week:** last lecture on open source + project time
- **Las week:** pelita tournament (deadline for player submission at 8pm the day before)

# The scientific python stack

  <img align="center" width="90%" src="http://fabienmaussion.info/acinn_python_workshop/figures/scipy_ecosystem.png"/>

## Pandas:  fast, flexible, and expressive data structures 

**Data from** https://data.oecd.org/agroutput/meat-consumption.htm 

In [ ]:
import pandas as pd
import io
import requests
import matplotlib.pyplot as plt
%matplotlib inline
 
url = ('https://stats.oecd.org/sdmx-json/data/DP_LIVE/.MEATCONSUMP.../OECD?'
       'contentType=csv&detail=code&separator=comma&csv-lang=en')
s=requests.get(url).content
df = pd.read_csv(io.StringIO(s.decode('utf-8')))

**The power of** [split-apply-combine](https://pandas.pydata.org/pandas-docs/stable/groupby.html)

In [ ]:
df.loc[(df.LOCATION == 'USA') & (df.MEASURE=='KG_CAP')].groupby('TIME').sum().Value.plot(label='USA');
df.loc[(df.LOCATION == 'EU28') & (df.MEASURE=='KG_CAP')].groupby('TIME').sum().Value.plot(label='EU28');
df.loc[(df.LOCATION == 'CHN') & (df.MEASURE=='KG_CAP')].groupby('TIME').sum().Value.plot(label='CHN');
df.loc[(df.LOCATION == 'IND') & (df.MEASURE=='KG_CAP')].groupby('TIME').sum().Value.plot(label='IND');
plt.legend();

In [ ]:
groups = df.loc[(df.LOCATION == 'USA') & (df.MEASURE=='KG_CAP')].groupby(['SUBJECT', 'TIME'])
groups['Value'].sum().unstack(level=0).plot();

## xarray: pandas for N-dimensional data 

## Our data

<img src="./figures/dataset.png" width="50%" align="right"> 

- numeric
- multi-dimensional
- labelled
- (lots of) metadata
- sometimes (very) large

## numpy.array

In [ ]:
import numpy as np
a = np.array([[1, 3, 9], [2, 8, 4]])
a

In [ ]:
a[1, 2]

In [ ]:
a.mean(axis=0) 

## xarray.DataArray

In [ ]:
import xarray as xr
da = xr.DataArray(a, dims=['lat', 'lon'], 
                  coords={'lon':[11, 12, 13], 'lat':[1, 2]})
da

In [ ]:
da.sel(lon=13, lat=2)

In [ ]:
da.mean(dim='lat')

# Arithmetics

## Broadcasting

<img src="./figures/broadcast.png" width="50%" align="left"> 

In [ ]:
a = xr.DataArray(np.arange(3), dims='time', 
                 coords={'time':np.arange(3)})
b = xr.DataArray(np.arange(4), dims='space', 
                 coords={'space':np.arange(4)})
a + b

## Alignment 

<img src="./figures/align.png" width="50%" align="left"> 

In [ ]:
a = xr.DataArray(np.arange(3), dims='time', 
                 coords={'time':np.arange(3)})
b = xr.DataArray(np.arange(5), dims='time', 
                 coords={'time':np.arange(5)+1})
a + b

# (Big) data: multiple files

Opening all files in a directory...

In [ ]:
mfs = '/home/mowglie/disk/Data/Gridded/GPM/3BDAY_sorted/*.nc'
dsmf = xr.open_mfdataset(mfs)

... results in a consolidated dataset ...

In [ ]:
dsmf

... on which all usual operations can be applied:

In [ ]:
dsmf = dsmf.sel(time='2015')
dsmf

Yes, even computations!

In [ ]:
ts = dsmf.precipitationCal.mean(dim=['lon', 'lat'])
ts

Computations are done "lazily"

No actual computation has happened yet:

In [ ]:
ts.data

But they can be triggered:

In [ ]:
ts = ts.load()
ts

In [ ]:
ts.plot();
ts.rolling(time=31, center=True).mean().plot();

# Thanks!